### 네이버 API 사용 가이드
- [공식사이트: 뉴스 검색 개발](https://developers.naver.com/docs/search/news/)
- [추가옵션부분 설명 블로그](https://blog.naver.com/ossiriand/221108706288)

### 정규표현식 참고 페이지
- [Advanced Regular Expression](https://www.python-course.eu/python3_re_advanced.php)
- [파이썬 정규표현식 모듈](http://devanix.tistory.com/296)

In [16]:
%%time
# page 를 for 문으로 돌리는 코드

import urllib.request
import requests
import json
import re
from time import sleep

client_key = 'V_8ikYRZIWV4ycuOFk1K'
client_secret = 'H6RcO1aP26'

target_ = 'news' #blog, news, etc...
encText = urllib.parse.quote_plus('한미약품 주가')
num_pages = 1000 #검색된 웹페이지의 순번. 1~1000
num_display = 100 #검색된 웹페이지에서 몇개까지 호출할 것인지. 10~100
sort_ = 'date' #정렬방법으로 유사도 또는 최신날짜순. sim | date


dtitle = []
ddate = []


for num in range(num_pages):
    
    naver_url = "https://openapi.naver.com/v1/search/{target}.json?query={query}&start={start}&display={display}&sort={sort}"
                .format(target=target_, query=encText, start= num+1 , display=num_display, sort=sort_) 

    request = urllib.request.Request(naver_url)

    header_params = {"X-Naver-Client-Id":client_key, "X-Naver-Client-Secret":client_secret}

    response = requests.get(naver_url, headers = header_params)

    if(response.status_code == 200):
        data = response.json()
    else:
        print('Error Code:{}'.format(response.status_code))
        
    for i in range(100):
        ddate.append(data['items'][i]['pubDate'])
        dtitle.append(data['items'][i]['title'])
    
    if num % 50 == 0:
        print('{}th crwaling was done.'.format(num))
    
    if num == num_pages - 1:
        print('Crwaling was finished!')
    
    sleep(2)


0th crwaling was done.
50th crwaling was done.
100th crwaling was done.
150th crwaling was done.
200th crwaling was done.
250th crwaling was done.
300th crwaling was done.
350th crwaling was done.
400th crwaling was done.
450th crwaling was done.
500th crwaling was done.
550th crwaling was done.
600th crwaling was done.
650th crwaling was done.
700th crwaling was done.
750th crwaling was done.
800th crwaling was done.
850th crwaling was done.
900th crwaling was done.
950th crwaling was done.
Crwaling was finished!
CPU times: user 18.9 s, sys: 1.18 s, total: 20.1 s
Wall time: 37min 29s


In [17]:

#크롤링한 내용 중에서 날짜와 제목만 저장함.

#title의 내용에 포함되어 있는 html 태그들을 re 로 제거함.
for i in range(len(dtitle)):
    dtitle[i] = re.sub('<b>', '', dtitle[i])
    dtitle[i] = re.sub('</b>', '', dtitle[i])
    dtitle[i] = re.sub('&quot;', '', dtitle[i])
    dtitle[i] = re.sub('amp;', '', dtitle[i])

In [18]:
#중복내용 제거, 순서 유지.
from collections import OrderedDict

originalList = dtitle
#list(OrderedDict.fromkeys(originalList))
dtitle_ordered = list(OrderedDict.fromkeys(originalList))

In [19]:
len(dtitle_ordered)

1066

In [20]:
%%time
#reference: https://stackoverflow.com/questions/17055117/python-json-dump-append-to-txt-with-each-variable-on-new-line
import json
with open('news_title_price_ordered.txt', 'w') as outfile:
    for line in dtitle_ordered:
        json.dump(line, outfile, ensure_ascii=False)
        outfile.write('\n')

CPU times: user 18.5 ms, sys: 2.35 ms, total: 20.8 ms
Wall time: 19.2 ms


In [21]:
with open('to_token.txt', 'w') as outfile:
    for line in dtitle_ordered:
        json.dump(line, outfile, ensure_ascii=False)

### 실수들
- for 문 안에서 naver_url 을 돌릴때, 자꾸 에러가 나서 많은 시간을 허비했다. 결과적으로 for 문은 0부터 시작하는데 start 파라미터는 1~1000 의 범위라서 요청과정에 에러가 났던 것. 
- 그래서 [ num + 1 ] 로 파라미터 입력을 바꿔주니까 문제 해결!!
- 비슷한 실수가 반복과정을 출력할때도 for 문을 100 으로 돌리면 0~99 로 나오는 것을 깜빡하고 finish 를 출력하면서 조건을 100으로 맞추니 출력이 되지 않았다. 
- 이러한 숫자체계에 익숙해져야 할 것이다. 

- for 문을 이중으로 돌릴때, 내부의 for 도 num 으로 돌리면 나중에 예기치 못한 문제가 생길 수도. 구분해서 사용하자. 

- 처리과정이 오래 걸리고, 만들면서 오류가 많아서 자꾸 꺼지는 코드는 진행상황을 모니터하기 위해서 중간중간에 print 를 해주는게 좋다. 이 역시 텀을 너무 많이 주지 말고, 적당한 간격으로 해야지 코드가 잘 작동하고 있는지 모니터하기 용이하다. 

- json 파일을 인덱싱하는 과정에서 어려움이 있었다. 
- 구체적으로 첫 크롤링하는 for문 안에서는 json 파일을 인덱싱(title, date)한 것을 리스트로 반환하고, 밖으로 나와서 이를 대상으로 다시 for문을 돌려서 전처리작업을 진행하였다. 
- 이유는 크롤링하는 과정에서 자꾸 오류가 나서, 1000 개를 돌리는게 서버에 무리가 가나? 안에서 전처리까지하는게 부담되나? 하는 걱정 때문이었다. 결론적으로는 아무 문제도 없었다.
- 하지만, 이런 걱정 때문에 밖으로 나와서 작업하였는데, json에서 인덱싱과 list에서의 인덱싱 방법에 차이가 있었다.
- 인덱싱을 편하게 하고자 dataframe으로 만들어서 처리하려고 했지만, list를 dataframe에 넣는 것도 문제가 좀 있었다. 
- 그래서 결국 다시 처음 디자인대로 첫크롤링코드에 json을 list로 반환하는 코드를 함께 넣어버렸다. 그러니까 잘 작동함.
- 결론적으로 json 이든, list 든, dataframe 이든 그 구조와 데이터타입, 그리고 인덱싱 방법 등에 대해서 알아둘 필요가 있다. 

- 정말 기가 막히게도 10000개의 내용을 중복을 거르고 나니까 198개 밖에 안남았다;;
- 중복 제거는 반드시 해야할 듯.

In [8]:
    
ddate_split=[]

#reference: https://www.python-course.eu/python3_re_advanced.php

for num in range(len(ddate)):
    temp_ = ddate[num].split(' ', 5)
    ddate_split.append(temp_)
#    ddate_split.append(ddate[num].split(' ', 5))

#date 의 구성요소 중에서 일, 월, 년, 시간 만 추출함.
for num in range(len(ddate_split)):
    ddate_split[num] = ddate_split[num][1:5]
    
# 장 마감시간 근처 이후에 올라온 기사는 1, 그 이전의 기사는 0.
for num in range(len(ddate_split)):
    if int(ddate_split[num][3].split(':')[0]) > 14:
        ddate_split[num][3] = 1
    else:
        ddate_split[num][3] = 0
    
#앞에서 작업한 일, 월, 년, 장마감시간기준을 각각 리스트로 반환함.
day = []
month = []
year = []
after_market = []

for num in range(len(ddate_split)):
    day.append(int(ddate_split[num][0]))
    month.append(ddate_split[num][1])
    year.append(int(ddate_split[num][2]))
    after_market.append(ddate_split[num][3])
    
#     if i % 100 == 0:
#         print('{}th append was done.'.format(i))
#     elif i == len(dtitle)-1:
#         print('ddate append was finished!')
    

IndentationError: expected an indented block (<ipython-input-8-276802f383ae>, line 7)